# Orbitales moleculares y curvas de energía potencial
# Parte 1: Moléculas diatómicas

Javier Cerezo, UAM (Madrid)</p>
Marzo 2021


## Inicialización

In [41]:
# Import modules
import nglview as nv
import ase.io
# Load psi4 to compute orbitals
import psi4
import qcelemental as qcel
# Interactive stuff (ipywidgets)
import ipywidgets as widgets
from ipywidgets import interactive, VBox, HBox, Output
# Matplotlib to make plots
%matplotlib widget
import matplotlib.pyplot as plt
# Numpy for matrix manipulation and scipy for interpolation
import numpy as np
from scipy.interpolate import interp1d
# datetime to get unique job id:
from datetime import datetime
from uuid import uuid4
# Manage files/folders
import os, shutil, glob

<font color='red'>**ATENCIÓN**</font>: 
Ejecuta la siguiente celda antes de iniciar la práctica. **NO** vuelvas a ejecutarla una vez iniciada.

In [42]:
# Get unique job_id
job_id = datetime.now().strftime('%Y%m%d%H%M%S') + '_' + str(uuid4())[:5]

# Create a folder with job_id name and change cwd to it
user = os.getenv("USER")
base_path = os.getcwd() + '/'
#os.chdir(base_path)
job_path = base_path + job_id + '/'
os.mkdir(job_path)
#os.chdir(job_path)

# NOTE:
# There is a conflict between file loading by ase/nv and os.chdir. So, chdir is not used
# along the note book. Instead, files generated in-place (e.g. cubefiles) are copied to 
# created folders. This implies that simultaneous run of this notebook may lead to 
# unexpected results in some cases (but rather unlikely)

## Datos de entrada

Los datos de entrada para un cálculo de estructura electrónica son:

* Posición de los núcleos atómicos
* Carga de cada núcleo (número atómico)
* Número total de electrones en el sistema
* Número de electrones desapareados (multiplicidad de espín)

Todos estos datos quedan definidos indicando la estructura molecular junto con la carga y multiplicidad.

Existen una gran cantidad de formatos estandarizados para escribir la geometría molecular. Entre ellos, uno de los más sencillos, es el formato `xyz`, en el que la primera línea indica el número de átomo, la segunda puede usarse para incluir una descripción y las siguientes contienen el elemento junto con las posiciones X,Y,Z del núcleo (en Angstrongs). Para moléculas complejas, podemos usar editores moleculares que permiten construir las moléculas gráficamente. Para moléculas sencillas, como las diatómicas, pueden escribirse estos ficheros "a mano". En el caso de una molécula diatómica, basta con especificar los elementos que corresponden a cada átomo y la distancia de enlace para construir la estructura. Indica esta información en el formulario que aparece al ejecutar la siguiente celda. Se generará la estrucutra en formato `xyz`.

In [3]:
def set_structure(at1,at2,dist):
    global fxyz, geomxyz
    
    if len(at1) == 0 or len(at2) == 0:
        return None
    
    
    if at1 not in qcel.periodictable.E:
        raise BaseException('Átomo desconocido: {}'.format(at1))
    if at2 not in qcel.periodictable.E:
        raise BaseException('Átomo desconocido: {}'.format(at2))
    
    if at1 == at2:
        mol_name = at1+'2'
    else:
        mol_name = at1 + at2
    
    comment='Molécula de {}'.format(mol_name)

    # Estructura molecular
    ## Formato: XYZ
    geomxyz = '''2
    {}
    {} 0.0 0.0 0.0
    {} 0.0 0.0 {:<8.3f}
    '''.format(comment,at1,at2,dist)
    print('------------------------------\n'+geomxyz)
    fxyz = job_path+'test.xyz'
    null = open(fxyz,'w').write(geomxyz)

interactive(set_structure,
            at1 = widgets.Text(value = 'F', description = 'Átomo 1'),
            at2 = widgets.Text(value = 'H', description = 'Átomo 2'),
            dist = widgets.FloatText(value = 1.00, step=0.1, description = 'Dist (Å)'))

interactive(children=(Text(value='F', description='Átomo 1'), Text(value='H', description='Átomo 2'), FloatTex…

Podemos visualizar la estructura resultante con una gran variedad de visores. En este documento interactivo se hace uso de uno de ellos. La siguiente celda muestra la estructura. Usa el ratón para acercar/alejar y rotar las molécula. Puedes mostrar la distancia de enlace presionando el sobre él con el botón derecho del ratón.

In [4]:
mol = ase.io.read(fxyz)
view1 = nv.show_ase(mol)
view1.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}
view1

NGLWidget()

Como hemos dicho anteriormente, para poder realizar un cálculo de estructura electrónica, necesitamos indicar, además de la estructura molecular, su carga total y la multiplicidad de espín. El fichero de entrada de un programa de cálculo electrónica debe contener estos datos, junto con la estructura.

Para esta práctica, solo vamos a realizar cálculos con estructuras singlete (`mult=1`). Ten en cuenta que no todas las cambinaciones de carga y multiplicidad son físicamente válidos. En el caso de 

En el caso de `Psi4`, el programa que vamos a usar en esta práctica, el fichero de entrada quedaría como indica la celda siguiente:

In [5]:
def set_mol(carga,mult):
    global geomxyz, psi4_mol
    
    # Generate input
    psi4_inp = geomxyz.split('\n')
    psi4_inp = '\n'.join(psi4_inp[2:])
    psi4_inp = '{} {}\n'.format(carga,mult) + psi4_inp
    # Show on screen
    print('----------------------------------\n'+psi4_inp)
    # Set psi4 Molecule
    try:
        psi4_mol = psi4.geometry(psi4_inp)
    except:
        raise BaseException('La combinación de carga {} y multiplicidad {} es imposible'.format(carga,mult))

cargas = [ str(i) for i in range(-2,3) ]
    
interactive(set_mol,
            carga = widgets.Combobox(value = '0', options = cargas, description = 'Carga',ensure_option=False),
            mult  = widgets.Dropdown(value = '1', options = ['1'], description = 'Mult', ensure_option=True))

interactive(children=(Combobox(value='0', description='Carga', options=('-2', '-1', '0', '1', '2')), Dropdown(…

## Cálculo electrónico

Un cálculo electrónico consiste en resolver la ecuación de Schrödinger electrónica:

\begin{equation}
\hat{H}_{el}\psi_{el}(\mathbf{r};\mathbf{R}) = E_{el}(\mathbf{R})\psi_{el}(\mathbf{r};\mathbf{R})
\label{Eq:Schr}
\end{equation}

Y la energía total del sistema debe incluir, además de $E_{el}$ la repulsión entre núcleos:

\begin{equation}
V(\mathbf{R}) = E_{el}(\mathbf{R}) + \sum_{i=1}^{i=n_{at}}\sum_{j=1}^{j<i}\frac{Z_iZ_j}{|R_i-R_j|}
\end{equation}

Sin embargo, la resolución de la ecuación (\ref{Eq:Schr}) solo es posible, de forma exacta, para la molécula de dihidrógeno (o moléculas/iones análogos, con un solo electrón). Para el resto, debemos emplear aproximaciones. Por esta razón, debemos de indicar al programa cuál es la aproximación que vamos a emplear. 

En esta práctica, vamos a emplear la teoría del funcional de la densidad (DFT, por sus siglas en inglés) como método para resolver el problema electrónico.

### Ajustes del cálculo

Aunque el método DFT es, en principio, exacta, en la práctica debemos de emplear un funcional aproximado entre la larga lista de funcionales desarrollados hasta la fecha. Elije un funcional de los propuestos en el menú desplegable que aparece al ejecutar la siguiente celda: 

In [6]:
avail_functionals = ['B3LYP','PBE0','WB97X']

def get_var(option):
    global funcional
    funcional = option
    return None

interactive(get_var,option = widgets.Dropdown(options = avail_functionals, description="Funcional: "))

interactive(children=(Dropdown(description='Funcional: ', options=('B3LYP', 'PBE0', 'WB97X'), value='B3LYP'), …

Asimismo, además de emplear un funcional aproximado, la resolución de las ecuaciones se realizar numéricamente, para lo que es necesario introducir un conjunto de funciones de base. Elije un conjunto de funciones de base de los propuestos en el menú desplegable que crea la siguiente celda:

In [7]:
avail_basis = ['6-31G(d)','6-31+G(d)','aug-cc-pVDZ']

def get_var(option):
    global base
    base = option
    return None

interactive(get_var,option = widgets.Dropdown(options = avail_basis, description="Funciones de base: "))

interactive(children=(Dropdown(description='Funciones de base: ', options=('6-31G(d)', '6-31+G(d)', 'aug-cc-pV…

### Energía en un punto y optimización

Ya tenemos todo lo neceario para realizar el cálculo. Nuestro método nos va a proporcionar una aproximación a la energía total del sistema y la función de onda. Realiza el cálculo ejecutando la celda siguiente (puede tardar varios segundos).

In [8]:
# Cálculo de la energía
E, wfn = psi4.energy(funcional+'/'+base,return_wfn=True)

print('Método de cálculo: {}/{}'.format(funcional,base))
print('Energía: {:8.3f} hartrees'.format(E))

dist = mol.get_distance(0,1)
print('Distancia de enlace (inicial): {:8.3f} Å'.format(dist))

Método de cálculo: B3LYP/6-31G(d)
Energía: -100.416 hartrees
Distancia de enlace (inicial):    1.000 Å


----------------

Como vemos, al terminar el cálculo, tenemos acceso a la energía del sistema. Además, obtenemos también una función de onda, que consiste en el producto de funciones monoelectrónicas: los **orbitales moleculares**, que vamos a analizar en la siguiente sección. 

Antes, vamos a obtener la estructura de equilibrio, es decir, aquella para la que la energía del sistema se minimiza. Como hemos indicado anteriormente, la energía del sistema es una función de la posición de los núcleos atómicos, $V(R)$. En este caso, será una función de la distancia del único enlace.

Para obtener la energía del mínimo temos que minimizar la energía (optimizar la función energía). Los programas de cálculo electrónico proporcionan métodos numéricos para realizar esta optimización.

In [9]:
# Optimización de la energía (estructura de equilibrio)
E, wfn = psi4.optimize(funcional+'/'+base,return_wfn=True)

print('Energía: {:8.3f} hartrees'.format(E))

# New structure to ase object
geomxyz = psi4_mol.to_string('xyz')
fxyz = job_path+'test.xyz'
null = open(fxyz,'w').write(geomxyz)
mol = ase.io.read(fxyz)

dist = mol.get_distance(0,1)
print('Distancia de enlace (equilibrio): {:8.3f} Å'.format(dist))

Optimizer: Optimization complete!
Energía: -100.420 hartrees
Distancia de enlace (equilibrio):    0.934 Å


A continuación, podemos mostrar la estructura optimizada ejecutando la celda siguiente.

In [10]:
mol = ase.io.read(fxyz)
view2 = nv.show_ase(mol)
view2.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}
view2

NGLWidget()

--------------

### Orbitales moleculares

En primer lugar debemos generar los ficheros que contienen el valor de cada función (orbital molecular) sobre una maya de puntos tridimensional. Usamos el formato `cube`, bastante extendido para guardar datos volumétricos.

In [11]:
# Create a new folder ('SinglePoint') to generate the cube files on
cube_folder = 'SinglePoint/'
cube_path = job_path + cube_folder
try:
    os.mkdir(cube_path)
except:
    shutil.rmtree(cube_path, ignore_errors=True)
    os.mkdir(cube_path)
#os.chdir(cube_path)
# Generate cubes
psi4.driver.p4util.cubeprop(wfn)
# Retunr
#os.chdir(job_path)

# Avoiding os.chdir, Seems to be the cause of some isses loading files
# Or maybe we simply need to provide always the full path (instead of
# relative path).
# We'll try to avoid chdir for now:
# The following step (glob search) can cause conflicts with other running
# notebooks if executed from both
cubefiles = glob.glob('Psi_*cube')
for cubefile in cubefiles:
    os.rename(cubefile,cube_path + cubefile)
os.rename('geom.xyz',cube_path + 'geom.xyz')

Una vez generados los ficheros en formato `cube` podemos representarlos. Antes de proceder a ello, vamos a escoger si deseamos que se muestren los orbitales internos.

In [12]:
# Get core orbitals
n_per_shell = [1,4,9,16,]
n_core  = 0
for Z in mol.numbers:
    p = qcel.periodictable.to_period(Z)
    n_core += np.array([ n_per_shell[i] for i in range(p-1) ],dtype=int).sum()

# Set interactive option
def get_var(option):
    global skip_core
    skip_core = option
    return None

interactive(get_var,
            option = widgets.Checkbox(value=True,description='Ignorar OM internos'))

interactive(children=(Checkbox(value=True, description='Ignorar OM internos'), Output()), _dom_classes=('widge…

La siguiente celda (escondida por defecto) recopila información relavante de los orbitales y los dibuja junto con el diagrama. Utiliza el menu desplegable que se genera al ejecutar la celda para inspeccionar los distintos orbitales. Se muestran los orbitales internos solo si se ha indicado en la casilla anterior. Si se cambia el valor de la casilla, vuelve a ejecutar la celda siguiente para ver los cambios

In [13]:
# ** ENERGIES **
# Get symms
pg = psi4_mol.point_group()
ct = pg.char_table()
irep_symbols = [ ct.gamma(i).symbol() for i in range(pg.order()) ]

# Get MO energies (per symm)
mo_symm_eners = wfn.epsilon_a().nph
    
# Get all in one array and sort
mo_all_eners = np.concatenate(mo_symm_eners)
inds = np.argsort(mo_all_eners)

# Get symm labels
mo_all_ireps = []
for i,mo_symm_ener in enumerate(mo_symm_eners):
    for j,ii in enumerate([i]*len(mo_symm_ener)):
        mo_all_ireps.append(str(j+1)+'-'+irep_symbols[ii])
mo_all_ireps = np.array(mo_all_ireps)

# Sort mo_ireps and mo_eners together
mo_all_ireps = mo_all_ireps[inds]
mo_all_eners = mo_all_eners[inds]
# Get last core index
i_core = n_core-1
mo_all_shell = ['C']*(i_core+1) + ['V']*(len(mo_all_eners)-i_core-1)

# Manage degenerancies
mo_diagram=[]
ener_last=99999.
for ener,irep,shell in zip(mo_all_eners,mo_all_ireps,mo_all_shell):
    if np.isclose(ener,ener_last,atol=0.001):
        for i in range(len(mo_diagram[-1])):
            mo_diagram[-1][-1][3] = 'D'+str(i+1)
        mo_diagram[-1].append([ener,irep,shell,'D'+str(i+2)])
    else:
        mo_diagram.append([[ener,irep,shell,'ND']])
    ener_last=ener
    
# Make a mapping from serial to with-degenerancies
all_to_diagram_map = []
for i,levels in enumerate(mo_diagram):
    for j,level in enumerate(levels):
        all_to_diagram_map.append([i,j])
        
# ** DIAGRAM (FIGURE) **
# WARNING: figures must be closed
out_diagram1 = Output(layout={'border': '1px solid black'})
with out_diagram1:
    OMdiagram1, ax1 = plt.subplots(1,figsize=(1.8,6))
    OMdiagram1.tight_layout()
    OMdiagram1.canvas.header_visible = False
    ax1.set_ylabel('Energy, a.u.')
    ax1.set_xticklabels([])

    for level in mo_diagram:
        # Skip core orbitals
        if level[0][2] == 'C' and skip_core:
            continue
        if len(level) == 1:
            ax1.hlines(level[0][0],xmin=1.6,xmax=2.4,color='k')
        else:
            ax1.hlines(level[0][0],xmin=1.5,xmax=1.9,color='k')
            ax1.hlines(level[1][0],xmin=2.1,xmax=2.5,color='k')

# Initialize orb/geo ids
if skip_core:
    i0  = i_core + 1
else:
    i0 = 0
    
# Get list of orbitals
orbital_list = []
k = 0
for i,j in all_to_diagram_map:
    orbital_list.append(str(k+1)+'_'+mo_diagram[i][j][1])
    k += 1
    
# Initialize (cont'd)
mo_ = orbital_list[i0]
iso_ = 0.02
repr_type_ = 'superficie'
highlighted = None
    
# Initialize view
cubefile = cube_path + 'Psi_a_'+orbital_list[i0]+'.cube'
mol = ase.io.read(cubefile)
view3 = nv.show_ase(mol)
view3.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}

# Store orbs surfare representations
orbs = []
# component_1 (store component address in orbs list)
orbs.append(view3.add_component(cubefile))
orbs[-1].clear()
orbs[-1].add_surface(opacity=0.5, color='blue', isolevelType="value", isolevel=0.02)
orbs[-1].add_surface(opacity=0.5, color='red',  isolevelType="value", isolevel=-0.02, depthWrite=False)
orbs[-1].hide()

# Build an output container to print info about orbital
out_label=Output()

# DISPLAY
    
def update_repr(mo,iso,repr_type):
    global view3, orbs, mo_, iso_, repr_type_, highlighted
    
    # Only one MO is loaded in memory (for all geoms)
    # So, if it changes, reload the right one
    if (mo != mo_ or iso != iso_ or repr_type != repr_type_):
        #Update mo (load new set)
        # Set is_wire
        if repr_type == 'superficie':
            is_wire = False
        else:
            is_wire = True
        # remove current
        for orb in orbs:
            view3.remove_component(orb)
        # load new mo
        orbs=[]
        cubefile = cube_path + 'Psi_a_'+mo+'.cube'
        # component_1 (store component address in orbs list)
        orbs.append(view3.add_component(cubefile))
        orbs[-1].clear()
        orbs[-1].add_surface(opacity=0.5, wireframe=is_wire, color='blue', isolevelType="value", isolevel=abs(iso))
        orbs[-1].add_surface(opacity=0.5, wireframe=is_wire, color='red',  isolevelType="value", isolevel=-abs(iso), depthWrite=False)
        orbs[-1].hide()
        
    # Display selected
    orbs[-1].show()
    
    # Update diagram
    k = int(mo.split('_')[0])-1
    i,j = all_to_diagram_map[k]
    ener = mo_diagram[i][j][0]
    if mo_diagram[i][j][3] == 'ND':
        x0, xf = 1.6, 2.4
    elif mo_diagram[i][j][3] == 'D1':
        x0, xf = 1.5, 1.9
    elif mo_diagram[i][j][3] == 'D2':
        x0, xf = 2.1, 2.5
    else:
        x0, xf = 2.6, 2.9
    if highlighted:
        highlighted.remove()
    highlighted = ax1.hlines(ener,xmin=x0,xmax=xf,linewidths=3,color='yellow', visible=True, alpha=0.7)
    
    # Update atom info
    out_label.clear_output()
    with out_label:
        print('MO: {}  |  E(hartree) = {:8.3f}'.format(mo,ener))
    
    # Update ids
    mo_ = mo
    repr_type_ = repr_type
    iso_ = iso
    
    
# DISPLAY
controls = interactive(update_repr, 
                       mo=widgets.Dropdown(options=orbital_list[i0:],
                                           value=orbital_list[i0],
                                           description='MO:'),
                       iso=widgets.FloatText(value = 0.02, step=0.01, description = 'Isovalor'),
                       repr_type=widgets.Dropdown(options=['superficie','malla'],
                                                   value='superficie',
                                                   description='Representación'))
surfbox = VBox([out_label,view3, controls],layout={'width': '700px'})
HBox([surfbox,out_diagram1])

--------------------------

## Barrido de la distancia de enlace

Como hemos comentado al inicio, a cada estrucutra molecular le corresponde un valor de energía potencial total. Es decir, el potencial es una función de la geometría de la molécula, definida por la posición relativa de los núcleos atómicos: $V(\mathbf{R})$. En general, llamaremos a esta función \textbf{superficie de energía potencial}, que dependerá del número de coordenadas internucleares. En el caso de moléculas diatómicas, donde la geometría queda definida con solo un parámetro (distancia de enlace), tendremos una curva de potencial monodimensional. 

En la sección anterior nos hemos fijado en la estrucutra para la que esa función es un mínimo (estructura de equilibrio). Este es un punto muy relevante de la superficie de energía potencial, pero conocer esta función más allá del mínimo nos permite entender cómo se mueven las moléculas (vibraciones, flexibilidad...) y analizar su reactividad (rotura y formación de enlaces). En esta práctica, vamos a analizar la curva de energía potencial para la molécula diatómica que estamos tratando. Para ello, calcularemos la energía (y orbitales moleculares) para distintos valores de la distancia de equilibrio.

Usa los formularios que se generan al ejecutar la siguiente celda para seleccionar el rango para el que vamos a realizar el barrido de la distancia de enlace.

In [14]:
def get_var(option1,option2,option3):
    global dmin, dmax, nstep
    dmin = option1
    dmax = option2
    nstep = option3
    
    print('\n')
    print('Parámetros para el barrido:\n----------------------------')
    print('Rmin  = {:5.1f}'.format(dmin))
    print('Rmax  = {:5.1f}'.format(dmax))
    print('Pasos = {:5}'.format(nstep))
    
    return None

interactive(get_var,
            option1 = widgets.FloatText(value = 0.80, description = 'Rmin (Å)'),
            option2 = widgets.FloatText(value = 2.50, description = 'Rmax (Å)'),
            option3 = widgets.BoundedIntText(value = 10, min=2, step=1, description = 'Pasos'))

interactive(children=(FloatText(value=0.8, description='Rmin (Å)'), FloatText(value=2.5, description='Rmax (Å)…

Una vez definido el rango a lo largo del que se hará el barrido, ejecuta la siguiente celda para realizar los cálculos

In [15]:
# First check funcional and base (not present if section 3 is skiped)
if 'funcional' not in globals():
    print('Selecciona un funcional y una base primero')
elif 'base' not in globals():
    print('Selecciona un funcional y una base primero')
else:

    # Clean cube directories first
    geom_folders = glob.glob(job_path+'GEOM*')
    for folder in geom_folders:
        shutil.rmtree(folder, ignore_errors=True)

    # Scan range
    distances = np.linspace(dmin,dmax,nstep)

    # Define interatomic unitary vector
    geom = psi4_mol.geometry().np
    v = geom[1]-geom[0]
    u = v/np.linalg.norm(v)

    # Initialize arrays to store data
    ener_scan = []
    dist_scan = []
    success_scan = []
    mo_symm_eners_scan = []

    # Prints to screen
    print("Energía total a lo largo del barrido\n\n")
    print("          R [Ang]                 E [hartree]       ")
    print("---------------------------------------------------------")


    for i,dist in enumerate(distances):
        # Generate new geom (better from xyz string with updated R)
        dist = np.round(dist,3)
        geom = psi4_mol.geometry().np
        geom[1] = geom[0] + u*dist/psi4.constants.bohr2angstroms
        geom=psi4.core.Matrix.from_array(geom)
        psi4_mol.set_geometry(geom)
        try: 
            E, wfn = psi4.energy(funcional+'/'+base,return_wfn=True)
            ener_scan.append(E)
            dist_scan.append(dist)
            success_scan.append(True)
            # Update table
            print("           {:5.3f}                  {:<1.6f}".format(dist, E))
            # Cubefiles
            geom_folder = job_path + 'GEOM_{:03g}'.format(i)+'/'
            os.mkdir(geom_folder)

            psi4.driver.p4util.cubeprop(wfn)

            cubefiles = glob.glob('Psi_*cube')
            for cubefile in cubefiles:
                os.rename(cubefile,geom_folder + cubefile)
            os.rename('geom.xyz',geom_folder + 'geom.xyz')
            # MO energies
            mo_symm_eners_scan.append(wfn.epsilon_a().nph)

        except:
            dist_scan.append(dist)
            ener_scan.append(None)
            success_scan.append(False)
            # Update table
            print("           {:5.3f}                  {:}".format(dist, 'ERROR'))
    print("---------------------------------------------------------\n")

Energía total a lo largo del barrido


          R [Ang]                 E [hartree]       
---------------------------------------------------------
           0.800                  -100.394368
           0.989                  -100.417434
           1.178                  -100.384222
           1.367                  -100.340090
           1.556                  -100.298136
           1.744                  -100.261856
           1.933                  -100.231401
           2.122                  -100.206564
           2.311                  -100.186540
           2.500                  -100.170480
---------------------------------------------------------



### Curva de energía

Una vez realizados los cálculos, podemos representar la curva de energía. Para ello, en lugar de usar directamente los datos de energía absoluta total en Hartrees, es más conveniente realizar la representación de la energía respecto a una geometría de referencia en unidades con más "sentido químico", como kcal/mol.

Ejecuta la siguiente celda y selecciona los parámetros de la representación.

In [16]:
out_curve = Output()
with out_curve:
    pes_fig, pes_plot = plt.subplots(1)
    pes_fig.tight_layout()
    pes_fig.canvas.header_visible = False
    pos1 = pes_plot.get_position() # get the original position 
    pos2 = [pos1.x0 + 0.1, pos1.y0 + 0.05,  pos1.width * 0.9, pos1.height * 0.95] 
    pes_plot.set_position(pos2) # set a new position
    
out_msg = Output()

def update_plot(is_relative,ref_geom,units):
    global dist_scan, ener_scan, success_scan
    
    out_msg.clear_output()
    
    # Relative geom
    if is_relative:
        i_ref = dist_scan.index(ref_geom)
        if not success_scan[i_ref]:
            with out_msg:
                print('ERROR: el cálculo falló a la geometría seleccionada. Dist = {:5.3f}'
                      .format(dist_scan[i_ref]))
            pes_plot.clear()
            return None
        Eref = ener_scan[i_ref]
    else:
        Eref = 0
    
    # Filter data (to discard failed jobs)
    ener_sucess = np.array(ener_scan)[np.array(success_scan)==True]
    dist_sucess = np.array(dist_scan)[np.array(success_scan)==True]
    dist_failed = np.array(dist_scan)[np.array(success_scan)==False]

    # Compute relative energy
    if units == 'kcal/mol':
        factor = qcel.constants.hartree2kcalmol
    elif units == 'kJ/mol':
        factor = qcel.constants.hartree2kJmol
    elif units == 'eV':
        factor = qcel.constants.hartree2ev
    elif units == 'cm-1':
        factor = qcel.constants.hartree2wavenumbers
    elif units == 'Hartree':
        factor = 1
    
    Eplot = (ener_sucess - Eref) * factor
    
    # Make interpolation (only possible if we have 4 data points at least)
    if len(Eplot)>3:
        interpo = interp1d(dist_sucess, Eplot, kind='cubic',fill_value='extrapolate')
    else:
        interpo = interp1d(dist_sucess, Eplot, kind='linear',fill_value='extrapolate')
    
    # Update plot
    # Sucessful points
    pes_plot.clear()
    pes_plot.set_ylabel('Energía ({})'.format(units))
    pes_plot.set_xlabel('R (Å)')
    pes_plot.plot(dist_sucess,Eplot,'ob')
    # Interpolation
    xdata = np.linspace(dist_sucess[0],dist_sucess[-1],100)
    pes_plot.plot(xdata,interpo(xdata),'--k')
    # Failed point (interpolated/extrapolated)
    pes_plot.plot(dist_failed,interpo(dist_failed),'xr')
    
    return None
    
    
    
units_list = ['kcal/mol','kJ/mol','Hartree','eV','cm-1']
controls = interactive(update_plot,
            is_relative = widgets.Checkbox(value = True, description='Energía relativa'),
            ref_geom = widgets.Dropdown(options = dist_scan, description = 'Ref. Geom. (Å)'),
            units = widgets.Dropdown(options = units_list, description = 'Unidades'))
VBox([controls,out_curve,out_msg])

### Orbitales moleculares

Por último, vamos a echar un vistazo a cómo cambian los orbitales a lo largo del barrido. Ejecuta la celda siguiente para obtener un menu interactivo para visualizar los orbitales. Ten en cuenta que, al seleccionar una orbital, se cargan los datos de este orbital para todas las geometrías, por lo que puede tarda unos segundos. La actualización al cambial la geometría es más fluída.

In [17]:
# Get core orbitals
n_per_shell = [1,4,9,16,]
n_core = 0
n_vale = 0
for Z in mol.numbers:
    p = qcel.periodictable.to_period(Z)
    n_core += np.array([ n_per_shell[i] for i in range(p-1) ],dtype=int).sum()
    n_vale += n_per_shell[p-1]

# Set interactive option
def get_var(option1,option2):
    global skip_core, only_vale
    skip_core = option1
    only_vale = option2
    return None

interactive(get_var,
            option1 = widgets.Checkbox(value=True,description='Ignorar OM internos'),
            option2 = widgets.Checkbox(value=True,description='Mostrar capa de valencia mínima'))

interactive(children=(Checkbox(value=True, description='Ignorar OM internos'), Checkbox(value=True, descriptio…

In [18]:
###############################
## ** TRAYECTORIA **
###############################
# Set traj view
## Name of intermediate file
trfile=job_path + 'scan.traj'
# From single sctructures to trajectory loaded on nglview
## Get structures into ase object
mols = []
mol_files = glob.glob(job_path + 'GEOM_*/geom.xyz')
mol_files.sort()
for file in mol_files:
    mols.append(ase.io.read(file))
## Write into traj file
ase.io.write(trfile,mols)
## Read trajectory intp ase and convert to nglview object
asetraj = ase.io.trajectory.TrajectoryReader(trfile)
## Generate view with trajectory
view4 = nv.show_asetraj(asetraj)
view4.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}
# Disable embeded player slider
view4.player.widget_player_slider.close()


###############################
## ** ORBITALES **
###############################
# Initialize ids (geo)
geo_ = 0


# 
# ** ENERGIES **
# 
# Get symmetry (irep) symbols (assume PG is not changing with geom)
pg = psi4_mol.point_group()
ct = pg.char_table()
irep_symbols = [ ct.gamma(i).symbol() for i in range(pg.order()) ]

# Get MO energies (per symm) at the selected geom
mo_symm_eners = mo_symm_eners_scan[geo_]
# Paste (concatenate) data for all symms
mo_all_eners = np.concatenate(mo_symm_eners)
mo_sym_ireps = [ np.array([i]*len(l)) for i,l in enumerate(mo_symm_eners) ]
mo_sym_ireps = [ np.array([ str(i+1)+'-'+irep_symbols[j] \
                           for i,j in enumerate(irep_symm)]) for irep_symm in mo_sym_ireps]
mo_all_ireps = np.concatenate(mo_sym_ireps)
inds = np.argsort(mo_all_eners)

# Sort mo_ireps and mo_eners together
mo_all_ireps = mo_all_ireps[inds]
mo_all_eners = mo_all_eners[inds]

#
# ** DIAGRAM (FIGURE) **
#
# WARNING: figures must be closed
out_diagram2 = Output(layout={'border': '1px solid black'})
with out_diagram2:
    OMdiagram2, ax2 = plt.subplots(1,figsize=(1.8,6))
    OMdiagram2.tight_layout()
    OMdiagram2.canvas.header_visible = False
    pos1 = ax2.get_position() # get the original position 
    pos2 = [pos1.x0 + 0.1, pos1.y0 + 0.05,  pos1.width * 0.9, pos1.height * 0.95] 
    pes_plot.set_position(pos2) # set a new position
    ax2.set_ylabel('Energy, a.u.')
    ax2.set_xticklabels([])

# 
# ** DRAW ORBS **
# 
# Initialize (cont'd)
if skip_core:
    i0  = n_core
else:
    i0 = 0
if only_vale:
    ilast = n_core + n_vale
else:
    ilast = len(mo_all_eners)
mo_ = mo_all_ireps[i0]
highlighted = None
iso_ = 0.02
repr_type_ = 'superficie'

    
# Get cubefiles
cube_files = glob.glob(job_path + 'GEOM_*/Psi_a_*_'+mo_+'.cube')
cube_files.sort()

# Initialize orbs surfare representations
orbs = []
for cubefile in cube_files:
    orbs.append(view4.add_component(cubefile))
    orbs[-1].clear()
    orbs[-1].add_surface(opacity=0.5, wireframe=False, color='blue', isolevelType="value", isolevel=abs(iso_))
    orbs[-1].add_surface(opacity=0.5, wireframe=False, color='red',  isolevelType="value", isolevel=-abs(iso_), depthWrite=False)
    orbs[-1].hide()

# Build an output container to print info about orbital
out_label=Output()

# DISPLAY
    
def update_repr(geo,mo,iso,repr_type):
    global view4, orbs, geo_, mo_, iso_, repr_type_, highlighted, mo_ener
    
    # Initialize to catch when data are not loaded
    have_cube_data = True
    
    # Only one MO is loaded in memory (for all geoms)
    # So, if it changes, reload the right one
    if (mo != mo_ or iso != iso_ or repr_type != repr_type_):
        out_label.clear_output()
        with out_label:
            print('Espera mientras se cargan los orbitales')
        #Update mo (load new set)
        # Set is_wire
        if repr_type == 'superficie':
            is_wire = False
        else:
            is_wire = True
        # remove current
        for orb in orbs:
            view4.remove_component(orb)
        # load new mo
        orbs=[]
        cube_files = glob.glob(job_path + 'GEOM_*/Psi_a_*_'+mo+'.cube')
        cube_files.sort()
        for cubefile in cube_files:
            orbs.append(view4.add_component(cubefile))
            orbs[-1].clear()
            orbs[-1].add_surface(opacity=0.5, wireframe=is_wire, 
                                 color='blue', isolevelType="value", isolevel=abs(iso))
            orbs[-1].add_surface(opacity=0.5, wireframe=is_wire, 
                                 color='red',  isolevelType="value", isolevel=-abs(iso), depthWrite=False)
            orbs[-1].hide()
            
    if (mo != mo_ or geo != geo_ or highlighted == None):
        # Get MO energies (per symm) at the selected geom
        mo_symm_eners = mo_symm_eners_scan[geo]
        # Paste (concatenate) data for all symms
        mo_all_eners = np.concatenate(mo_symm_eners)
        mo_sym_ireps = [ np.array([i]*len(l)) for i,l in enumerate(mo_symm_eners) ]
        mo_sym_ireps = [ np.array([ str(i+1)+'-'+irep_symbols[j] \
                                   for i,j in enumerate(irep_symm)]) for irep_symm in mo_sym_ireps]
        mo_all_ireps = np.concatenate(mo_sym_ireps)
        inds = np.argsort(mo_all_eners)

        # Sort mo_ireps and mo_eners together
        mo_all_ireps = mo_all_ireps[inds]
        mo_all_eners = mo_all_eners[inds]

        # ** DIAGRAM (FIGURE) **
        # WARNING: figures must be closed
        ax2.clear()
        ax2.set_ylabel('Energy, a.u.')
        ax2.set_xticklabels([])

        # Manage degenerancies
        levels = []
        ener_ = 99999.
        for ener,symm in zip(mo_all_eners[i0:ilast],mo_all_ireps[i0:ilast]):
            if np.isclose(ener,ener_,atol=0.01):
                levels[-1].append([ener,symm])
            else:
                levels.append([[ener,symm]])
            ener_=ener
            
        have_cube_data = False
        hl0 = None
        for level in levels:
            if len(level) == 1:
                ax2.hlines(level[0][0],xmin=-0.5,xmax=0.5,color='k')
                if level[0][1] == mo:
                    ener, hl0, hlf = level[0][0], -0.5, 0.5
                    
            elif len(level) == 2:
                ax2.hlines(level[0][0],xmin=-0.65,xmax=-0.15,color='k')
                if level[0][1] == mo:
                    ener, hl0, hlf = level[0][0], -0.65, -0.15
               #    
                ax2.hlines(level[1][0],xmin=0.15,xmax=0.65,color='k')
                if level[1][1] == mo:
                    ener, hl0, hlf = level[1][0], 0.15, 0.65
            else:
                # Fix ini-fin, and fit levels within
                # Settings
                sep_space = 0.02
                lev_ini = -0.7
                lev_fin = 0.7
                #
                sep_spaces = sep_space*(len(level)-1)
                lev_space = (lev_fin-lev_ini-sep_spaces)/len(level)
                l0_ = lev_ini - sep_space
                for i in range(len(level)):
                    l0 = l0_ + sep_space
                    lf = l0  + lev_space
                    ax2.hlines(level[i][0],xmin=l0,xmax=lf,color='k')
                    if level[i][1] == mo:
                        ener, hl0, hlf = level[i][0], l0, lf
                    l0_ = lf
        
        if hl0 is not None:
            have_cube_data = True
            # Highlight selected
            if highlighted:
                highlighted.remove()
            highlighted = ax2.hlines(ener,xmin=hl0,xmax=hlf,
                                     linewidths=3,color='yellow', visible=True, alpha=0.7)
        mo_ener = ener
        
    #Update geo
    # Geom
    view4.frame = geo
    # Orbs
    orbs[geo_].hide()
        
    # Update atom info
    out_label.clear_output()
    if have_cube_data:
        # Display selected
        orbs[geo].show()
        with out_label:
            print('MO: {}  |  E(hartree) = {:8.3f}'.format(mo,mo_ener))
    else:
        with out_label:
            print('MO: {}  |  No hay datos'.format(mo))
    

    
    # Update ids
    mo_=mo
    geo_=geo
    repr_type_=repr_type
    iso_=iso
    
    
# DISPLAY
controls = interactive(update_repr,
                       geo=widgets.IntSlider(value=0,min=0,max=len(cube_files)-1,description='Geom'),
                       mo=widgets.Dropdown(options=list(mo_all_ireps[i0:ilast]),
                                           value=mo_all_ireps[i0],
                                           description='MO:'),
                       iso=widgets.FloatText(value = 0.02, step=0.01, description = 'Isovalor'),
                       repr_type=widgets.Dropdown(options=['superficie','malla'],
                                                   value='superficie',
                                                   description='Representación'))
surfbox = VBox([out_label,view4, controls],layout={'width': '700px'})
HBox([surfbox,out_diagram2])



## Borrar datos

Al finalizar la práctica, pulsa sobre el botón que se genera a ejecutar la siguiente celda para borrar los datos. Es posible que tengas que pulsar dos veces (el botón debe "responder" que ha borrado los datos si los había).

¡Aseguraté de haber guardado los datos que te vaya a hacer falta!

In [44]:
n_clicks = 0

out_borrar = Output()

import time

def remove_data(opt):
    
    global job_path, n_clicks
    
    n_clicks += 1
    out_borrar.clear_output()
    
    # Since the function is called with the cell
    # we need to skip the action the first time
    # its called
    if n_clicks>1:
        n_clicks = 0
        if os.path.exists(job_path):
            shutil.rmtree(job_path, ignore_errors=True)
            with out_borrar:
                print('Datos borrados')
            time.sleep(2)
            out_borrar.clear_output()
        else:
            with out_borrar:
                print('No hay datos que borrar')
            time.sleep(2)
            out_borrar.clear_output()
    
    return None

controls = interactive(remove_data,
            opt = widgets.ToggleButton(description = 'Borrar datos',
                                       icon = 'remove'))
VBox([controls, out_borrar])